In [1]:
import numpy as np
import opendssdirect as dss
import matplotlib.pyplot as plt
import pandas as pd
import re
import sys
import time
import scipy.linalg as spla
from lib.compute_KCL_matrices import compute_KCL_matrices
from lib.basematrices import basematrices
from lib.NR3_timevarying import NR3_timevarying

In [2]:
fn = '06node_threephase_unbalancedVR.dss'
dss.run_command('Redirect ' + fn)
print(dss.Lines.AllNames())
print(dss.Circuit.AllBusNames())

['line_sub_a01', 'line_a01_a02_in', 'line_a01_a02_out', 'line_a02_a03', 'line_a03_a04', 'line_a03_a05', 'line_a05_a06']
['sourcebus', 'a01', 'a02', 'a03', 'a04', 'a05', 'a06']


In [3]:
dss.Lines.AllNames()
dss.Circuit.AllBusNames()

def linelist(busname):
        in_lines = np.array([])
        out_lines = np.array([])
        for k in range(len(dss.Lines.AllNames())):
            dss.Lines.Name(dss.Lines.AllNames()[k])
            if busname in dss.Lines.Bus1():
                out_lines = np.append(out_lines, dss.Lines.AllNames()[k])
            elif busname in dss.Lines.Bus2():
                in_lines = np.append(in_lines, dss.Lines.AllNames()[k])
        return in_lines,out_lines
linelist(dss.Circuit.AllBusNames()[2])

(array(['line_a01_a02_in', 'line_a01_a02_out'], dtype='<U32'),
 array(['line_a02_a03'], dtype='<U32'))

Vectorized Solver

In [2]:
slackidx = 0
Vslack = np.array([1, np.exp(1j*-120*np.pi/180), np.exp(1j*120*np.pi/180)])

fn = 'compare_opendss_05node_threephase_unbalanced_oscillation_03.dss'
#fn = '06node_threephase_radial_unbalanced.dss'
#fn = 'IEEE_13Node_Modified_01.dss'
fn = '06node_threephase_meshed_unbalanced.dss'
fn = '06node_threephase_unbalancedVR.dss'
#fn = '02node_threephase_unbalanced_new.dss'

# Time-steps
times = np.linspace(0, 2*np.pi, 5)
# DER changes over time-steps
der = [0, 0, 0, 0, 0]
# Capacitance changes over time-steps
capacitance = [0, 0, 0, 0, 0]

# Pre-generate matrices
t00 = time.time()
X, g_SB, b_SB, G_KVL, b_KVL, H, g, b, H_reg, G_reg = basematrices(fn, slackidx, Vslack, None, None)

t0 = time.time()
for _ in range(1):   
    dss.run_command('Redirect ' + fn)
    nnode = len(dss.Circuit.AllBusNames())
    nline = len(dss.Lines.AllNames())

    VNR01 = np.zeros((len(times), 3, nnode), dtype = "complex")
    INR01=  np.zeros((len(times), 3, nline), dtype = "complex")
    STXNR01 = np.zeros((len(times), 3, nline), dtype = "complex")
    SRXNR01 = np.zeros((len(times), 3, nline), dtype = "complex")
    iNR01 = np.zeros((len(times), 3, nnode), dtype = "complex")
    sNR01 = np.zeros((len(times), 3, nnode), dtype = "complex")

    for i in range(len(times)):
        # Run NR3 with variations in time, DER, and capacitance
        VNR, INR, STXNR, SRXNR, iNR, sNR, itercount = \
            NR3_timevarying(fn, X, g_SB, b_SB, G_KVL, b_KVL, H, g, b, None, None, der[i], capacitance[i], times[i], H_reg, G_reg)    
        VNR01[i, :, :] = np.reshape(VNR, (3, nnode))
        INR01[i, :, :] = np.reshape(INR, (3, nline))
        STXNR01[i, :, :] = np.reshape(STXNR, (3, nline))
        SRXNR01[i, :, :] = np.reshape(SRXNR, (3, nline))
        iNR01[i, :, :] = np.reshape(iNR, (3, nnode))
        sNR01[i, :, :] = np.reshape(sNR, (3, nnode))
t1 = time.time()

Iteration number 0.000000
Iteration number 1.000000
Iteration number 2.000000
Iteration number 3.000000
Iteration number 4.000000
Iteration number 5.000000
Iteration number 6.000000
Iteration number 7.000000
VNR
[[ 1.        +0.00000000e+00j  0.99748364-8.63045054e-04j
   1.01743331-8.80305955e-04j  1.01483667+2.81409576e-03j
   1.0160466 +2.84574972e-03j  1.01146262+4.91384318e-03j
   0.        +0.00000000e+00j]
 [-0.5       -8.66025404e-01j -0.50098929-8.69807314e-01j
  -0.51100907-8.87203461e-01j -0.50923092-8.90807903e-01j
   0.        +0.00000000e+00j -0.51036173-8.92383545e-01j
  -0.5102562 -8.91918369e-01j]
 [-0.5       +8.66025404e-01j -0.50300643+8.60776827e-01j
  -0.51306656+8.77992363e-01j -0.51757143+8.73746443e-01j
  -0.51759915+8.71148756e-01j  0.        +0.00000000e+00j
   0.        +0.00000000e+00j]]
INR:
[[ 4.48274969e-01-0.10337166j  4.43841700e-01-0.10235135j
   4.48274969e-01-0.10337166j  3.45576493e-01-0.0776947j
   9.84888242e-02-0.02432932j  1.48536741e-01-0.0487

Iteration number 7.000000
Iteration number 8.000000
VNR
[[ 1.        +0.j          0.99737436-0.00089644j  1.01732185-0.00091437j
   1.01464201+0.00293196j  1.01589869+0.00296583j  1.01114277+0.00511718j
   0.        +0.j        ]
 [-0.5       -0.8660254j  -0.50104218-0.86995917j -0.51106302-0.88735835j
  -0.50921512-0.89111397j  0.        +0.j         -0.51038607-0.89275165j
  -0.51027308-0.89226847j]
 [-0.5       +0.8660254j  -0.50313692+0.86058081j -0.51319965+0.87779243j
  -0.5178808 +0.87339428j -0.51790896+0.87069652j  0.        +0.j
   0.        +0.j        ]]
INR:
[[ 4.65626302e-01-0.10808412j  4.61021664e-01-0.10701742j
   4.65626302e-01-0.10808412j  3.58966772e-01-0.08140622j
   1.02294645e-01-0.02525659j  1.54308487e-01-0.05056993j
   7.16870507e-28+0.j        ]
 [ 5.49215225e-02-0.17386534j -2.38419835e-01-0.27147235j
   5.49215225e-02-0.17386534j -1.65837557e-01-0.19624685j
  -6.77723602e-26+0.j         -1.10594277e-01-0.11127974j
  -3.85575587e-02-0.03614177j]
 [-3.868820

In [ ]:
print('allocate base matrices ', t0 - t00)
print('run newton rapshon 20x ', t1-t0)
print(t1-t00)

OpenDSS Solution

In [2]:
t0_dss = time.time()
for _ in range(1):
    dss.run_command('Redirect compare_opendss_05node_threephase_unbalanced_oscillation_03.dss')
    #dss.run_command('Redirect 06node_threephase_unbalanced.dss') 
    #dss.run_command('Redirect IEEE_13Node_Modified_01.dss')
    dss.run_command('Redirect 06node_threephase_unbalancedVR.dss')
    #dss.run_command('Redirect 02node_threephase_unbalanced_new.dss')

    VDSS0 = np.zeros((len(times), 3, nnode), dtype = "complex")
    IDSS0 =  np.zeros((len(times), 3, nline), dtype="complex")
    SRXDSS0 = np.zeros((len(times), 3, nline), dtype="complex")
    STXDSS0 = np.zeros((len(times), 3, nline), dtype="complex")
    load_arr = np.zeros((len(times), 3, nnode), dtype='complex')
    kW_list = np.array([])
    kvar_list = np.array([])

    # Store the loads
    for k in range(len(dss.Loads.AllNames())):
        dss.Loads.Name(dss.Loads.AllNames()[k])
        kW_list = np.append(kW_list, dss.Loads.kW())
        kvar_list = np.append(kvar_list, dss.Loads.kvar())

    for i in range(len(times)):
        dss.Circuit.SetActiveBus(dss.Circuit.AllBusNames()[0])
        Vbase = dss.Bus.kVBase() * 1000
        Sbase = 1000000.0
        Ibase = Sbase/Vbase
        Zbase = Vbase/Ibase

        # Set slack bus (sourcebus) voltage reference in p.u.
        SlackBusVoltage = 1.000
        dss.Vsources.PU(SlackBusVoltage)

        # Time-varying load
        for k in range(len(dss.Loads.AllNames())):
            dss.Loads.Name(dss.Loads.AllNames()[k])
            dss.Loads.kW(kW_list[k]* (1 + 0.1*np.sin(2*np.pi*0.01*times[i])))
            dss.Loads.kvar(kvar_list[k] * (1 + 0.1*np.sin(2*np.pi*0.01*times[i])))

        dss.Solution.Convergence(0.000000000001)

        # Solve power flow with OpenDSS file
        dss.Solution.Solve()
        if not dss.Solution.Converged:
            print('Initial Solution Not Converged. Check Model for Convergence')
        else:
            print('Initial Model Converged. Proceeding to Next Step.')
            #Doing this solve command is required for GridPV, that is why the monitors
            #go under a reset process
            dss.Monitors.ResetAll()

            # set solution Params
            #setSolutionParams(dss,'daily',1,1,'off',1000000,30000)
            dss.Solution.Mode(1)
            dss.Solution.Number(1)
            dss.Solution.StepSize(1)
            dss.Solution.ControlMode(-1)
            dss.Solution.MaxControlIterations(1000000)
            dss.Solution.MaxIterations(30000)
            # Easy process to get all names and count of loads, a trick to avoid
            # some more lines of code
            TotalLoads=dss.Loads.Count()
            AllLoadNames=dss.Loads.AllNames()
            print('OpenDSS Model Compliation Done.')
            print('Iterations: ', dss.Solution.Iterations())
            print('Tolerance: ', dss.Solution.Convergence())

        print('')

        # OpenDSS loads
        for k in range(len(dss.Loads.AllNames())):
            dss.Circuit.SetActiveElement('Load.'+dss.Loads.AllNames()[k])
            pattern =  r"(\w+)\."
            load_bus = re.findall(pattern, dss.CktElement.BusNames()[0])
            idxbs = dss.Circuit.AllBusNames().index(load_bus[0]) 
            load_ph_arr_temp = [0, 0, 0] 
            for ph1 in range(1, 4): 
                pattern = r"\.%s" % (str(ph1))
                load_ph = re.findall(pattern, dss.CktElement.BusNames()[0])
                if load_ph:
                    load_ph_arr_temp[ph1 - 1] = 1
            n= 0
            for m in range(len(load_ph_arr_temp)):
                if load_ph_arr_temp[m] == 1:
                    load_arr[i, m, idxbs] +=  (dss.CktElement.Powers()[n] + 1j * dss.CktElement.Powers()[n + 1])*1e3/1e6     #where 1e6 is sbase 
                    n += 2

        print(dss.Solution.Converged())

        # Print number of buses, and bus names
        print(len(dss.Circuit.AllBusNames()))
        print(dss.Circuit.AllBusNames())

        # Print number of loads, and load names
        print(len(dss.Loads.AllNames()))
        print(dss.Loads.AllNames())

        print('')

        VDSS = np.zeros((3,len(dss.Circuit.AllBusNames())),dtype='complex')

        for k1 in range(len(dss.Circuit.AllBusNames())):

            dss.Circuit.SetActiveBus(dss.Circuit.AllBusNames()[k1])

        #     print(dss.Circuit.AllBusNames()[k1])
        #     print(dss.Bus.Nodes())

        #     print('puVOTLAGES - LN CARTESIAN')
        #     print(dss.Bus.PuVoltage())

            ph = np.asarray(dss.Bus.Nodes(),dtype='int')-1

            Vtemp = np.asarray(dss.Bus.PuVoltage())

            Vtemp = Vtemp[0:nnode*2-1:2] + 1j*Vtemp[1:nnode*2:2]


        #     print(np.asarray(dss.Bus.Nodes(),'int'))

            VDSS[ph,k1] = Vtemp    

        #     VDSS[np.asarray(dss.Bus.Nodes(),'int'),k1] = np.array(dss.Bus.PuVoltage()[0:5:2] + 1j*dss.Bus.PuVoltage()[1:6:2])


        #     VDSS[dss.Bus.Nodes()-1,k1] = dss.Bus.PuVoltage()[0:2:5]
        #     for k2 in range(len(dss.Bus.Nodes())):
        #         VDSS[int(dss.Bus.Nodes()[k2])-1,k1] = dss.Bus.PuVoltage()[2*k2] + 1j*dss.Bus.PuVoltage()[2*k2+1]

        print('VDSS\n', np.round(VDSS,decimals=6))


        IDSS = np.zeros((3,len(dss.Lines.AllNames())),dtype='complex')

        for k1 in range(len(dss.Lines.AllNames())):
            dss.Lines.Name(dss.Lines.AllNames()[k1])
        #     print(dss.Lines.AllNames()[k1])
            ph = np.asarray(dss.CktElement.BusNames()[0].split('.')[1:], dtype='int')-1
            Imn = np.asarray(dss.CktElement.Currents())/Ibase
        #     print(Imn)
            Imn = Imn[0:int(len(Imn)/2)]
        #     print(Imn)
            Imn = Imn[0:(len(ph)*2)-1:2] + 1j*Imn[1:len(ph)*2:2]
        #     print(Imn)
            IDSS[ph,k1] = Imn
        #     print('')

        print('IDSS\n', np.round(IDSS,decimals=6))

        STXDSS = np.zeros((3,len(dss.Lines.AllNames())),dtype='complex')
        SRXDSS = np.zeros((3,len(dss.Lines.AllNames())),dtype='complex')

        for k1 in range(len(dss.Lines.AllNames())):
            dss.Lines.Name(dss.Lines.AllNames()[k1])
            ph = np.asarray(dss.CktElement.BusNames()[0].split('.')[1:], dtype='int')-1
            Sk = np.asarray(dss.CktElement.Powers())/(Sbase/1000)

            STXtemp = Sk[0:int(len(Sk)/2)]
            SRXtemp = Sk[int(len(Sk)/2):]

            STXtemp = STXtemp[0:len(ph)*2-1:2] + 1j*STXtemp[1:len(ph)*2:2]
            SRXtemp = -(SRXtemp[0:len(ph)*2-1:2] + 1j*SRXtemp[1:len(ph)*2:2])

            STXDSS[ph,k1] = STXtemp
            SRXDSS[ph,k1] = SRXtemp

        VDSS0[i, :, :] = VDSS
        IDSS0[i, :, :] = IDSS
        SRXDSS0[i, :, :] = SRXDSS
        STXDSS0[i, :, :] = STXDSS


        print('STXDSS\n', np.round(STXDSS,decimals=6))
        print('SRXDSS\n', np.round(SRXDSS,decimals=6))

        print('|VDSS|\n', np.round(np.abs(VDSS),decimals=6))
        print('<VDSS\n', np.round(180/np.pi*np.angle(VDSS),decimals=6))
        print('D<VDSS\n', 180/np.pi*np.angle(VDSS) - 180/np.pi*np.angle(VDSS[:,[0]]))
t1_dss = time.time()

NameError: name 'times' is not defined

In [ ]:
VDSS0

Comparing results between three solutions - DSS, 01 (vectorized), 02 (nonvectorized)

In [ ]:
for i in range(len(times)):
    print("Timestep %d \n" % i)
    print('Complex Voltage difference')
    #print(np.max(np.abs(VNR01[i] - VNR02[i])))
    print(np.max(np.abs(VDSS0[i] - VNR01[i]))) 

    print('Complex Current difference')
    print(np.max(np.abs(IDSS0[i] - INR01[i])))
    
    print('Complex TX Power difference')
    print(np.max(np.abs(STXDSS0[i] - STXNR01[i])))
    
    print('Complex RX Power difference')
    print(np.max(np.abs(SRXDSS0[i] - SRXNR01[i])))
    
    print('Voltage Magnitude difference')
    print(np.max(np.abs(np.abs(VDSS0[i]) - np.abs(VNR01[i]))))
    
    print('Voltage Angle difference')
    print(np.max(np.abs(180/np.pi*np.angle(VDSS0[i]) - 180/np.pi*np.angle(VNR01[i]))))
    print("\n")

In [ ]:
np.abs(VDSS0[0]-VNR01[0])
print(np.abs(VNR01[0]))

Load comparison between non-vectorized and OpenDSS

In [ ]:
for i in range(len(times)):
    print("Timestep %d \n Magnitude of the Difference: \n" % i)
    print(np.abs(spah[i] - load_arr[i]))
    print("Max difference: \n ")
    print(np.max(np.abs(spah[i] - load_arr[i])))
    print("\n")

In [ ]:
dss.run_command('Redirect IEEE_13Node_Modified_01.dss')
for i in range(len(dss.Loads.AllNames())):
    dss.Loads.Name(dss.Loads.AllNames()[i])
    print(dss.Loads.kW())
    print(dss.Loads.kvar())
    print(dss.CktElement.Powers())

---

In [ ]:
t = -1
def load_order_f():
    load_order = {}
    for n in range(len(dss.Loads.AllNames())):
        dss.Loads.Name(dss.Loads.AllNames()[n])
        pattern =  r"(\w+)\."
        load_bus = re.findall(pattern, dss.CktElement.BusNames()[0])
        if load_bus[0] not in load_order:
            load_order[load_bus[0]] = 1
        elif load_bus[0] in load_order:
            load_order[load_bus[0]] += 1
    return load_order
load_order_list = load_order_f()

def load_values():
    load_ph_arr = np.zeros((nnode, max(load_order_list.values()), 3))
    load_kw_arr_ph = np.zeros((3, nnode))
    load_kvar_arr_ph = np.zeros((3, nnode))
    if t == -1:
        var = 1
    else:
        var = (1 + 0.1*np.sin(2*np.pi*0.01*t))
    for load in range(len(dss.Loads.AllNames())):
        dss.Loads.Name(dss.Loads.AllNames()[load])
        load_data = dss.CktElement.BusNames()[0].split('.')
        idxbs = dss.Circuit.AllBusNames().index(load_data[0])
        for ph in range(1, len(load_data)):
            load_kw_arr_ph[int(load_data[ph]) - 1, idxbs] += dss.Loads.kW() * 1e3 / Sbase / (len(load_data) - 1)
            load_kvar_arr_ph[int(load_data[ph]) - 1, idxbs] += dss.Loads.kvar() * 1e3 / Sbase / (len(load_data) - 1) 
    return load_kw_arr_ph, load_kvar_arr_ph
load_kw_arr_ph, load_kvar_arr_ph = load_values()
print(load_kw_arr_ph)
print(load_kvar_arr_ph)

In [ ]:

            idxbs = dss.Circuit.AllBusNames().index(cap_data[0])
            for ph in range(1, len(cap_data)):
                caparr[int(cap_data[ph]) - 1, idxbs] += dss.Capacitors.kvar() * 1e3 / Sbase / (len(cap_data) - 1)
        return caparr

In [ ]:
[[0.         0.         0.10384609 0.10384609 0.10384609 0.15576913
  0.        ]
 [0.         0.         0.10384609 0.10384609 0.         0.10384609
  0.05192304]
 [0.         0.         0.10384609 0.10384609 0.10384609 0.
  0.        ]]
[[0.         0.         0.02596152 0.02596152 0.02596152 0.05192304
  0.        ]
 [0.         0.         0.02596152 0.02596152 0.         0.02596152
  0.02596152]
 [0.         0.         0.02596152 0.02596152 0.02596152 0.
  0.        ]]